In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import product


In [155]:
a = list(product(np.linspace(0,1,11).round(1),np.linspace(0,1,11).round(1),np.linspace(0,1,11).round(1)))
len(a)
rtable = {}


(1.0, 1.0, 1.0)

In [143]:
dopamine_b1 = np.zeros(1331)
for i,kk in enumerate(a):
    dopamine_b1[i] = np.round(2*kk[0]+0.6*kk[1]+0.1*kk[2],2)

dopamine_b2 = np.zeros(1331)
for i,kk in enumerate(a):
    dopamine_b2[i] = np.round(5*kk[0]+0.1*kk[1]+0.2*kk[2],2)
    
dopamine_b3 = np.zeros(1331)
for i,kk in enumerate(a):
    dopamine_b3[i] = np.round(1*kk[0]+0.1*kk[1]+0.01*kk[2],2)


In [160]:

max_dop = np.zeros(len(a))

for ii,ai in enumerate(a):
    rtable[ai,'bacteria_1'] = dopamine_b1[ii]
    rtable[ai,'bacteria_2'] = dopamine_b2[ii]
    rtable[ai,'bacteria_3'] = dopamine_b3[ii]
    max_dop[ii] = dopamine_b1[ii] +dopamine_b2[ii]+dopamine_b3[ii]

    
max(max_dop),max_dop[-1],max_dop

(9.11, 9.11, array([0.  , 0.03, 0.06, ..., 9.05, 9.08, 9.11]))

In [145]:
b =(tuple(np.random.choice(np.linspace(0,1,11),3).round())),np.random.choice(['bacteria_1','bacteria_2','bacteria_3'])

b,rtable.get(b)

(((0.0, 0.0, 1.0), 'bacteria_2'), 0.2)

### observations
1. When action can be nothing, it slowl

3. Possible actions in one nutrient only includes 0.1 (say, a regular increase in the consumption of alcohol. The probability of consuming that nutrient becomes very high along with highest possible reward state; and potentially reducing the craving or eating probability of other nutrients especially (at least one of them).


In [178]:
class Bacteria():
    def __init__(self,t):
        self.population_size = np.random.randint(100,200)
        self.type = t

class Organism():
    
    def __init__(self,n):
        self.nutrients = n
        self.brain = Brain(self)
        self.gut = [Bacteria('bacteria_'+str(i+1)) for i in range(3)]
       
        
    def run(self):
        self.brain.setState()
        for i in range(100000):
            self.brain.takeAction()
        
act = list(product(np.linspace(0,0.1,2),np.linspace(-0.1,0.1,2),np.linspace(-0.1,0.1,2)))
# act.remove((0.0,0.0,0.0))
act = tuple(act)

    
class Brain():
    
    def __init__(self,organism):
        self.organism = organism
#          check what to name this 
        self.actions = act
        self.state = (0,0,1)
        
        self.alpha = 1
        self.gamma = 0.1
        self.epsilon = 0.01
        self.qtable = {}

    def setState(self):
        self.state = self.organism.nutrients
        

    def takeAction(self):
        all_q = [self.qtable.get((self.state,a),0.0) for a in self.actions]
        all_max_ind = np.where(all_q == np.max(all_q))[0]
        action = self.actions[np.random.choice(all_max_ind)]
        
#         use exploration
        if np.random.random()< self.epsilon:
            action = tuple(np.random.choice([0.1,0.0,-0.1], 3))
            
            
        self.ostate = self.state
        self.oaction = action
        self.state = tuple(np.round(np.clip(np.array(self.ostate) + np.array(action),0,1),1))

        # this is the summation of dopamine from all bacterium, say         
        self.oreward = np.sum([rtable.get((self.ostate,b.type)) for b in self.organism.gut])
#         print(""" While selecting an action: 
#         old state = {0}
#         action taken = {1}
#         new state = {2}

#         reward = {3}
#         """.format(self.ostate, action,self.state,self.oreward,self.state))
        
        self.learn()
        
        
    def learn(self):
        old_q = self.qtable.get((self.ostate,self.oaction),0.0)
#         print("""
#         old_q  = {0}
#         self state = {4}, old state = {5}
#         qtable = {3}
#         """.format(old_q,len(self.actions),self.actions,
#                   self.qtable.get((self.state,self.actions[0]),np.nan),
#                    self.state,self.ostate))

        max_new_q = np.max([self.qtable.get((self.state,a),0.0) for a in self.actions])
        
        
        newq = old_q + self.alpha*(self.oreward + self.gamma*max_new_q - old_q)
        
        self.qtable[(self.ostate,self.oaction)] = np.round(newq,3)
        
        
#         print("""
#                 Old Q = {0}
#         max New Q = {1}
#         New Q = {2}

#         Q table = {3}
#     """.format(old_q,max_new_q,newq,self.qtable))

o = Organism((0.2,0.4,0.3))
o.run()
o.brain.qtable

{((0.2, 0.4, 0.3), (0.1, -0.1, 0.1)): 2.01,
 ((0.3, 0.3, 0.4), (0.1, 0.1, 0.1)): 2.76,
 ((0.4, 0.4, 0.5), (0.0, 0.1, 0.1)): 3.68,
 ((0.4, 0.5, 0.6), (0.0, 0.1, -0.1)): 3.79,
 ((0.4, 0.6, 0.5), (0.1, 0.1, 0.1)): 3.83,
 ((0.5, 0.7, 0.6), (0.1, -0.1, 0.1)): 4.75,
 ((0.6, 0.6, 0.7), (0.0, -0.1, -0.1)): 5.5,
 ((0.6, 0.5, 0.6), (0.0, -0.1, -0.1)): 5.39,
 ((0.6, 0.4, 0.5), (0.0, -0.1, 0.1)): 5.27,
 ((0.6, 0.3, 0.6), (0.1, 0.1, 0.1)): 5.23,
 ((0.7, 0.4, 0.7), (0.0, -0.1, -0.1)): 6.14,
 ((0.7, 0.3, 0.6), (0.0, -0.1, -0.1)): 6.03,
 ((0.7, 0.2, 0.5), (0.1, 0.1, 0.1)): 5.91,
 ((0.8, 0.3, 0.6), (0.0, -0.1, 0.1)): 6.83,
 ((0.8, 0.2, 0.7), (0.0, 0.1, 0.1)): 6.78,
 ((0.8, 0.3, 0.8), (0.0, -0.1, 0.1)): 6.89,
 ((0.8, 0.2, 0.9), (0.1, 0.1, -0.1)): 6.84,
 ((0.9, 0.3, 0.8), (0.1, -0.1, 0.1)): 7.69,
 ((1.0, 0.2, 0.9), (0.0, 0.1, -0.1)): 8.44,
 ((1.0, 0.3, 0.8), (0.0, 0.1, -0.1)): 8.49,
 ((1.0, 0.4, 0.7), (0.1, -0.1, -0.1)): 8.54,
 ((1.0, 0.3, 0.6), (0.0, -0.1, -0.1)): 8.43,
 ((1.0, 0.2, 0.5), (0.1, 0.1, -0.

In [174]:
from collections import OrderedDict
oo = OrderedDict(o.brain.qtable)
oo.keys()

odict_keys([((0.2, 0.4, 0.3), (0.1, 0.1, 0.1)), ((0.3, 0.5, 0.4), (0.1, -0.1, 0.1)), ((0.4, 0.4, 0.5), (0.1, -0.1, 0.1)), ((0.5, 0.3, 0.6), (-0.1, 0.1, 0.1)), ((0.4, 0.4, 0.7), (0.1, -0.1, 0.1)), ((0.5, 0.3, 0.8), (-0.1, 0.1, 0.1)), ((0.4, 0.4, 0.9), (-0.1, -0.1, -0.1)), ((0.3, 0.3, 0.8), (-0.1, 0.1, 0.1)), ((0.2, 0.4, 0.9), (0.1, 0.1, 0.1)), ((0.3, 0.5, 1.0), (-0.1, -0.1, -0.1)), ((0.2, 0.4, 0.9), (0.0, -0.1, 0.1)), ((0.2, 0.3, 1.0), (-0.1, 0.1, 0.1)), ((0.1, 0.4, 1.0), (-0.1, -0.1, -0.1)), ((0.0, 0.3, 0.9), (0.1, -0.1, 0.1)), ((0.1, 0.2, 1.0), (-0.1, -0.1, 0.1)), ((0.0, 0.1, 1.0), (-0.1, -0.1, -0.1)), ((0.0, 0.0, 0.9), (0.1, -0.1, -0.1)), ((0.1, 0.0, 0.8), (0.1, 0.1, 0.1)), ((0.2, 0.1, 0.9), (-0.1, 0.1, 0.1)), ((0.2, 0.1, 0.9), (-0.1, -0.1, -0.1)), ((0.2, 0.1, 0.9), (0.0, -0.1, -0.1)), ((0.2, 0.0, 0.8), (-0.1, -0.1, 0.1)), ((0.1, 0.0, 0.9), (-0.1, -0.1, -0.1)), ((0.0, 0.0, 0.8), (0.1, -0.1, -0.1)), ((0.1, 0.0, 0.7), (0.1, -0.1, -0.1)), ((0.2, 0.0, 0.6), (-0.1, -0.1, -0.1)), ((0.1, 0.

In [175]:
max_value = max(o.brain.qtable.values())


[k for k,v in o.brain.qtable.items() if v == max_value],max_value

([((1.0, 1.0, 1.0), (0.0, 0.0, 0.1)),
  ((1.0, 1.0, 1.0), (0.0, 0.0, 0.0)),
  ((1.0, 1.0, 1.0), (0.1, 0.1, 0.1)),
  ((1.0, 1.0, 1.0), (0.1, 0.0, 0.1)),
  ((1.0, 1.0, 1.0), (0.1, 0.0, 0.0)),
  ((1.0, 1.0, 1.0), (0.0, 0.1, 0.0)),
  ((1.0, 1.0, 1.0), (0.0, 0.1, 0.1)),
  ((1.0, 1.0, 1.0), (0.1, 0.1, 0.0))],
 10.122)

In [149]:
act = list(product(np.linspace(0,0.1,2),np.linspace(-0.1,0.1,2),np.linspace(-0.1,0.1,2)))
# act.remove((0.0,0.0,0.0))
act = tuple(act)
act

((0.0, -0.1, -0.1),
 (0.0, -0.1, 0.1),
 (0.0, 0.1, -0.1),
 (0.0, 0.1, 0.1),
 (0.1, -0.1, -0.1),
 (0.1, -0.1, 0.1),
 (0.1, 0.1, -0.1),
 (0.1, 0.1, 0.1))

In [2]:
class QLearn():
    def __init__(self):
        self.alpha = 0.2
        self.gamma = 0.7
        self.epsilon = 0.1
        self.qtable = {}
        self.agent = None
        self.ostate = ()
        self.oaction = 0
        
    def getState(self):
        return self.agent.state
        
    def getBestAction(self):
        all_q = [self.qtable.get((self.getState(),a),0.0) for a in self.agent.actions]
        all_max_ind = np.where(all_q == np.max(all_q))[0]
        action = self.agent.actions[np.random.choice(all_m_ind)]

        print(f"""
        possible q for all actions = {all_q}
        index of all max = {all_max_ind}
        action = {action_taken}
        """)
        
        self.ostate = self.getState()
        self.oaction = action
        return action
        
    def learn(self):
        old_q = self.qtable.get((self.ostate,self.oaction),0.0)
        max_new_q = np.max([self.qtable.get((self.getState(),a),0.0) for a in self.agent.actions])

        newq = old_q + self.alpha*(self.agent.reward + self.gamma*max_new_q - old_q)
        self.qtable[(self.ostate,self.oaction)] = newq
        
        

In [89]:
# global clock


class Organism():
    def __init__(self):
        self.categories = ['a','b','c']        
        self.gut = [Bacteria(c,self) for c in self.categories]
#         self.eatingBehavior - somehow this shud reflect on different nutrients intake 

        


In [37]:
# class Bacteria():
#     def __init__(self,cat, org):
#         self.category = cat
#         self.organism = org
#         self.population = np.random.randint(100,200)
# #         self.chemical = output(cat)
        
#     def output(self,cat):
#         chem = 0
#         return chem

## breakdown of steps

1. create a world (person) with 5 (5 is becoming unmanageable for this simple example) gut bacteria, and 5 nutrients?
2. [x] mapping between nutrients and bacteria - and that between bacteria and chemicals. make a table for now. 
3. when we initialise the person, the hunger signal tells the person to eat - with some probability ..
4. Nutrients get injected into the system - and once that is done, RL tells us what action to take .. 
    * state is identified with say, time of the day, gut bacterial population and thus, which will be reflected in the amount of dopamine that will come as a reward
    * amount of dopamine (instead of just presence)can be used as the possible reward state. 
    * action is the increase/decrease in the probability of getting a certain type of nutrients. 
    * for this use case, we inject a major proportion of alcohol or sugar into the system.
    * for now, we can also decide the initial composition of the nutrients and gut bacteria to test the use case in the most abstract form ?
    
  

In [87]:
from collections import defaultdict
d = defaultdict(dict)

# this way is not extensible at all, but for the momemnt - just stick with it to develop a simple use case,

d['carbs']['a'] = 0.1
d['carbs']['b'] = 1
d['carbs']['c'] = 0.2

d['fats']['a'] = 0.1
d['fats']['b'] = 1
d['fats']['c'] = 0





defaultdict(dict,
            {'carbs': {'a': 0.1, 'b': 1, 'c': 0.2},
             'fats': {'a': 0.1, 'b': 1, 'c': 0.2}})

In [80]:
# # use dictionary instead if dopamine remains constant and only that chemical brings in reward

# a = [['carbs','a','dopamine',1],
#      ['carbs','b','dopamine',3],
#      ['carbs','c','dopamine',1],
#      ['carbs','d','dopamine',0],
#      ['carbs','e','dopamine',0],
#      ['protein','a','dopamine',0],
#      ['protein','b','dopamine',0],
#      ['protein','c','dopamine',0],
#      ['protein','d','dopamine',0],
#      ['protein','e','dopamine',0],
#      ['fats','a','dopamine',0],
#      ['fats','b','dopamine',0],
#      ['fats','c','dopamine',0]
    
    
#     ]

# tab = pd.DataFrame(columns = {'nutrients','bacterium','chemical','quantity'},index = range(len(a)))

# for i,f in enumerate(a):
#     tab.iloc[i] = a[i]
    
# tab


,nutrients,quantity,chemical,bacterium
0,carbs,a,dopamine,1
1,carbs,b,dopamine,3
2,carbs,c,dopamine,1
3,carbs,d,dopamine,0
4,carbs,e,dopamine,0
5,protein,a,dopamine,0
6,protein,b,dopamine,0
7,protein,c,dopamine,0
8,protein,d,dopamine,0
9,protein,e,dopamine,0
